In [239]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline


In [240]:
words = open('/content/name.txt','r').read().splitlines()

In [241]:
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [242]:
len(words)

32033

In [243]:
# build the vocabulary of characters and mapping to from integers

cahrs = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(cahrs)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [244]:
# build the dataset

block_size = 3 # context length: how many characters do we take to
X,Y = [],[]
for w in words:
  print(w)
  context = [0]*block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join(itos[i] for i in context), '----->',itos[ix])
    context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

Streaming output truncated to the last 5000 lines.
doron
dovi
draysen
dredyn
dugan
durell
ebin
edwardo
efrem
ehaan
ehab
eirik
eldar
eliav
eligio
eloi
eluzer
elway
emaad
emeka
enki
ero
erron
esaiah
estiven
eulalio
everet
everly
evert
evian
eyas
ezer
ezren
faaris
facundo
famous
farzan
felton
field
filipe
finbar
finch
flynt
fordham
fouad
gabryel
garnet
gaurik
gavi
geovonni
germany
gerrard
gevorg
giuliani
graden
gradyn
grayer
grayton
greylen
grimm
gurshaan
hameed
hanzel
hasib
haylen
henning
herminio
hill
hiyan
hrihaan
huxson
hykeem
ihsaan
imer
imir
iremide
isahi
isaih
ishak
ishank
issachar
issam
issiah
iyaan
izai
izick
jabbar
jabri
jaccob
jadarrius
jadin
jaecion
jaelin
jaelon
jaevon
jah
jahcere
jahmeer
jahvon
jahzir
jaidin
jaishawn
jakaree
jakorey
jalan
jameis
jansiel
janthony
janziel
jarom
jaseer
jaston
jatavion
jathniel
javarius
jawuan
jaxel
jaxxyn
jaxzon
jayace
jaydien
jayke
jaymere
jayniel
jayonni
jayston
jayziah
jazen
jazion
jeramie
jeramyah
jerek
jeremey
jeren
jeric
jerman
jermani
je

In [245]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [246]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [247]:
Y

tensor([ 5, 13, 13,  ..., 26, 24,  0])

First we need to make the lookup table C

In [248]:
C = torch.randn((27,2))

In [249]:
C

tensor([[ 0.1180,  0.9406],
        [ 0.2678,  0.7175],
        [-0.1934, -0.4018],
        [-1.4675,  1.1864],
        [-1.5211,  0.0993],
        [ 0.2780, -0.6977],
        [ 0.2646, -0.2550],
        [ 0.5914, -0.7993],
        [-1.0386, -0.7824],
        [-0.5601,  0.1808],
        [ 0.9070,  0.5225],
        [ 1.0114,  0.1190],
        [ 0.3956, -0.2686],
        [ 0.8701,  0.3181],
        [-0.4191,  0.2926],
        [ 0.7169,  0.2757],
        [-0.3070,  0.0722],
        [ 1.6049, -1.0311],
        [-1.2411, -1.3240],
        [ 0.9507,  1.1495],
        [-0.8139, -1.0443],
        [-1.5358, -0.9180],
        [ 0.1623,  1.0544],
        [-1.4822,  1.3339],
        [ 0.4451,  0.3869],
        [-1.7090,  0.0037],
        [-0.0703,  2.1768]])

In [250]:
C[5]

tensor([ 0.2780, -0.6977])

In [251]:
F.one_hot(torch.tensor(5),num_classes=27).float() @ C


tensor([ 0.2780, -0.6977])

In [252]:
C[5]

tensor([ 0.2780, -0.6977])

In [253]:
C[[5,6,7]]

tensor([[ 0.2780, -0.6977],
        [ 0.2646, -0.2550],
        [ 0.5914, -0.7993]])

In [254]:
C[X]

tensor([[[ 0.1180,  0.9406],
         [ 0.1180,  0.9406],
         [ 0.1180,  0.9406]],

        [[ 0.1180,  0.9406],
         [ 0.1180,  0.9406],
         [ 0.2780, -0.6977]],

        [[ 0.1180,  0.9406],
         [ 0.2780, -0.6977],
         [ 0.8701,  0.3181]],

        ...,

        [[-0.0703,  2.1768],
         [-0.0703,  2.1768],
         [-1.7090,  0.0037]],

        [[-0.0703,  2.1768],
         [-1.7090,  0.0037],
         [-0.0703,  2.1768]],

        [[-1.7090,  0.0037],
         [-0.0703,  2.1768],
         [ 0.4451,  0.3869]]])

In [255]:
C[X].shape

torch.Size([228146, 3, 2])

In [256]:
X[13,2]

tensor(1)

In [257]:
C[X][13,2]

tensor([0.2678, 0.7175])

In [258]:
C[1]

tensor([0.2678, 0.7175])

In [259]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [260]:
W1 = torch.rand((6,100))
b1 = torch.rand(100)


In [261]:
# emb @ W1 + b
emb.view(-1,6)@ W1 + b1

tensor([[2.2263, 2.0711, 1.7564,  ..., 2.3770, 3.1083, 2.1276],
        [2.2285, 1.4701, 1.5462,  ..., 1.3541, 1.8838, 1.4153],
        [1.3453, 1.8938, 0.4426,  ..., 1.0017, 2.0256, 1.2160],
        ...,
        [2.4252, 1.1164, 3.0536,  ..., 2.6256, 2.4986, 2.4650],
        [0.2241, 1.7459, 0.6669,  ..., 2.7488, 3.3517, 2.3166],
        [2.6827, 1.8956, 0.8269,  ..., 2.6126, 2.9368, 0.8221]])

In [262]:
torch.cat([emb[:,0,:], emb[:,1,:], emb[:,2,:]],1).shape

torch.Size([228146, 6])

In [263]:
torch.cat(torch.unbind(emb,1),1).shape

torch.Size([228146, 6])

In [264]:
a = torch.arange(18)

In [265]:
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

As long as the multiplipcation of the number is the same, it will work

In [266]:
a.view(2,9)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [267]:
a.view(3,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [268]:
a.storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [269]:
h = emb.view(emb.shape[0],6) @ W1 + b1

In [270]:
h.shape

torch.Size([228146, 100])

In [271]:
h = torch.tanh(emb.view(-1,6)@ W1 + b1)

In [272]:
h

tensor([[0.9770, 0.9687, 0.9421,  ..., 0.9829, 0.9960, 0.9720],
        [0.9771, 0.8996, 0.9131,  ..., 0.8750, 0.9548, 0.8886],
        [0.8729, 0.9557, 0.4158,  ..., 0.7623, 0.9658, 0.8385],
        ...,
        [0.9845, 0.8063, 0.9956,  ..., 0.9896, 0.9866, 0.9857],
        [0.2204, 0.9409, 0.5829,  ..., 0.9918, 0.9975, 0.9807],
        [0.9907, 0.9559, 0.6788,  ..., 0.9893, 0.9944, 0.6762]])

In [273]:
h.shape

torch.Size([228146, 100])

In [274]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)


In [275]:
logits = h@W2 + b2

In [276]:
logits.shape

torch.Size([228146, 27])

In [277]:
counts = logits.exp()
prob = counts/counts.sum(1,keepdim=True)

In [278]:
prob.shape

torch.Size([228146, 27])

In [279]:
prob[0].sum()

tensor(1.)

In [280]:
# loss = -prob[torch.arange(32),Y].log().mean()

In [281]:
# loss

In [282]:
# -------------- now made respectable --------------


In [283]:
X.shape , Y.shape #dataset

(torch.Size([228146, 3]), torch.Size([228146]))

In [284]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2),generator=g)
W1 = torch.randn((6,100),generator=g)
b1 = torch.randn(100,generator=g)
W2 = torch.randn((100,27),generator=g)
b2 = torch.randn(27,generator=g)
parameters = [C,W1,b1,W2,b2]

In [285]:
sum(p.nelement() for p in parameters)

3481

In [286]:
for p in parameters:
  p.requires_grad = True

In [288]:
for _ in range(10):
  # forward pass
  emb = C[X] #(32,3,2)
  h = torch.tanh(emb.view(-1,6) @ W1 + b1) # (32,100)
  logits = h @ W2 + b2 # (32,27)
  # counts = logits.exp()
  # prob = counts/counts.sum(1,keepdim=True)
  # loss = -prob[torch.arange(32),Y].log().mean()
  loss = F.cross_entropy(logits,Y)
  print(loss.item())

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  for p in parameters:
    p.data += -0.1 * p.grad


14.002603530883789
13.253260612487793
12.57991886138916
11.983101844787598
11.47049331665039
11.051856994628906
10.709586143493652
10.407632827758789
10.127808570861816
9.864365577697754


In [291]:
torch.randint(0,X.shape[0],(32,))

tensor([ 31850, 135080,  38723,  37472, 221334, 122496, 165728, 123807,  57324,
         43137, 222977,  28609,  92316, 119138,  58850, 157425, 147665, 133323,
        189686,  92834,  24786, 201216, 222708,  34036, 122439,   1390,  79018,
         84595,  70143, 148131, 119778, 128863])

In [ ]:
s